<a href="https://colab.research.google.com/github/DeepLabCut/Primer-MotionCapture/blob/master/COLAB_Primer_MotionCapture_Fig3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motion Capture with Deep Learning: 
A Primer on Motion Capture with Deep Learning: Principles, Pitfalls and Perspectives

[Alexander Mathis](https://github.com/AlexEMG) | [Steffen Schneider](https://github.com/stes) | [Jessy Lauer](https://github.com/jeylau) | [Mackenzie Mathis](https://github.com/MMathisLab)


- https://github.com/DeepLabCut/Primer-MotionCapture

- How you can use data augmentation to train neural networks:
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1593721822684-DJJIOEEC4YHRO2ZRW6Z3/ke17ZwdGBToddI8pDm48kMqyqP2xgoLMxs8NG4McAT9Zw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZamWLI2zvYWH8K3-s_4yszcp2ryTI0HqTOaaUohrI8PIT5G0e3JcWjRHcSN0vw_Zk3UAC_JV7w-s1xTX3wfPLu0/augs.png?format=1000w)

This script uses Imgaug to display various augmentation methods 
for a few labeled images of birds (Data recorded with Alex' iphone 
at Aiguille du Midi near Mont Blanc)

Uses Imgaug:
Code: https://github.com/aleju/imgaug
Docs: https://imgaug.readthedocs.io/en/latest/index.html




Please click on the "play' button to run the code in each cell.


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

Import the dependencies:

In [ ]:
# Use TensorFlow 1.x:
%tensorflow_version 1.x
import os
os.environ["DLClight"]="True"
import deeplabcut
import tensorflow as tf
vers = (tf.__version__).split(".")
if int(vers[0]) == 1 and int(vers[1]) > 12:
    TF = tf.compat.v1  # behaves differently before 1.13
else:
    TF = tf

TF.logging.set_verbosity(TF.logging.ERROR)
DEBUG = True and "DEBUG" in os.environ and os.environ["DEBUG"]
from deeplabcut import DEBUG

import pandas as pd
import os
import numpy as np

import imgaug as ia
import imgaug.augmenters as iaa
from imgaug.augmentables import Keypoint, KeypointsOnImage
from deeplabcut.utils.auxfun_videos import imread, imresize
import imageio 

TensorFlow 1.x selected.
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Let's grab some demo data!

In [ ]:
# Clone the Primer repo so we can use the demo data:
!git clone -l -s git://github.com/DeepLabCut/Primer-MotionCapture.git cloned-DLC-Primer

%cd cloned-DLC-Primer
!ls

Setting up some variables:

In [ ]:
scale=.2
ia.seed(1)
#parameters for plotting:
color=(400,0,0)
size=7
alpha=.01


## Loading data:



In [ ]:
imfolder='montblanc_images'
Dataframe = pd.read_hdf(os.path.join(imfolder,"CollectedData_Daniel.h5"))

scorer=Dataframe.columns.get_level_values(0)[0]
individuals=Dataframe.columns.get_level_values(1)
bodyparts=Dataframe.columns.get_level_values(2)
Dataframe.head()

scorer            Daniel                            ...                       
individuals        bird1                            ...    bird8              
bodyparts           head                      tail  ... leftwing rightwing    
coords                 x            y            x  ...        y         x   y
img0155.png    76.844318  1068.722389    91.885294  ...      NaN       NaN NaN
img0719.png   527.883814   798.424352   527.883814  ...      NaN       NaN NaN
img0382.png  1002.648118   448.081409  1016.503454  ...      NaN       NaN NaN
img1164.png          NaN          NaN          NaN  ...      NaN       NaN NaN
img0071.png   322.057817   350.433043   338.709137  ...      NaN       NaN NaN

[5 rows x 64 columns]

# Let's look at some Augmentations! 

*this takes a few minutes: please check the "augmentation examples" folder for the completed files!

In [ ]:
#setting up augmentations
Augmentations=[]

augtype='rotateandscale'
#rotate & scale 
seq = iaa.Sequential([
    iaa.Multiply((1.2, 1.5)), # change brightness, doesn't affect keypoints
    iaa.Affine(
        rotate=23,
        scale=(0.9, 1.1)
    ) # rotate by exactly 23 deg and scale to 90-10%, affects keypoints
])
Augmentations.append([augtype, seq])

augtype='fog'
seq = iaa.Sequential([iaa.Fog()])
Augmentations.append([augtype,seq])

augtype='snow'
seq = iaa.Sequential([iaa.Snowflakes(flake_size=(.2,.5),density=(0.005, 0.07), speed=(0.01, 0.05))])
Augmentations.append([augtype,seq])        

augtype='GuassianBlur'
seq = iaa.Sequential([iaa.GaussianBlur(sigma=(0, 3.0))])
Augmentations.append([augtype,seq]) 

for ind, imname in enumerate(Dataframe.index):
        image=imresize(imread(os.path.join('montblanc_images',imname)),size=scale)
        ny,nx,nc=np.shape(image)

        kpts=[]
        for i in individuals:
            for b in bodyparts:
                x, y=Dataframe.iloc[ind][scorer][i][b]['x'], Dataframe.iloc[ind][scorer][i][b]['y']
                if np.isfinite(x) and np.isfinite(y):
                    kpts.append(Keypoint(x=x*scale,y=y*scale))

        kps=KeypointsOnImage(kpts, shape=image.shape)
        
        cells=[]

        # image with keypoints before augmentation 
        image_before = kps.draw_on_image(image, color=color,size=size,alpha=alpha)
        cells.append(image_before)
        
        for name, seq in Augmentations:
            image_aug, kps_aug = seq(image=image, keypoints=kps)
            image_after = kps_aug.draw_on_image(image_aug, color=color,size=size,alpha=alpha)
            cells.append(image_after)
    
        grid_image = np.hstack(cells)  # Horizontally stack the images
        imageio.imwrite('augmentationexamples/'+str(imfolder)+'_'+imname.split('.png')[0]+'_joint.png', grid_image)

After (or while) the code above runs, you can go look into the "augmentationexamples folder to see the images. In Colab, you can double click and they will open on the right side of this screen!

# Play around! What happens if you set the `augtype` to something else/or add more?

Check out the options in imgaug: https://imgaug.readthedocs.io/en/latest/index.html



In [ ]:
#For example, try adding this in the cell above, at line 23:
augtype='GuassianBlur'
seq = iaa.Sequential([iaa.GaussianBlur(sigma=(0, 3.0))])
Augmentations.append([augtype,seq]) 


![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1593748127529-O7LN1VK99YAKL6EU6MHY/ke17ZwdGBToddI8pDm48kDXDnTgl1slZNt80fPstB8hZw-zPPgdn4jUwVcJE1ZvWQUxwkmyExglNqGp0IvTJZamWLI2zvYWH8K3-s_4yszcp2ryTI0HqTOaaUohrI8PI7YJEkB-NQMg7QT3XqyKhWHKvgVbDU7y_mPhOqssYa08/montblanc_images_img0155_joint.png?format=1000w)


- Can you find out all the augmentations used in the main Figure?

- Want to use this in DLC, check out these suggestions: https://github.com/DeepLabCut/DeepLabCut/wiki/Data-Augmentation
